# KnitHAT 

This notebook is step three in the process of building [KnitHAT](http://knithat.works), which recommends hat patterns from Ravelry based on user-uploaded images. KnitHAT was my Insight Data Science fellowship project, and was built in only three weeks.

This code scrapes an image for each hat pattern from Ravelry, based on a list of pattern IDs provided.

In [43]:
import pandas as pd
import requests as rq
import re
import time
import pickle

### Things to change with each new CSV:
change filename and check off when done: 
- 1000 X re-running with better except clause
- 3000 X
- 5000 X will have errors because couldn't get rid of " on end of some URLs
- 7000 X
- 9000 X
- 11000 X but filenames are screwed up due to how I removed .0 -- all 0s were removed 
- 12000 X
- 16000 * big file *

Change value in following cell for which_file to the number in the filename

In [60]:
# read csv of pattern data into dataframe

d = pd.read_csv('/Users/ssivek/Insight_Projects/Ravelry_Project/Data/patterns12000.csv')

In [61]:
which_file = 12000 # enter numbers from csv filename here to use in path name for downloads

In [62]:
d.head(5)

,patterns,pattern_id,pattern_name,medium2_urls,author_name
0,"{""patterns"": {""17016"":{""comments_count"":0,""cre...",17016,13505 Bonnet,https://images4-e.ravelrycache.com/uploads/mam...,Dale Design
1,"{""patterns"": {""541305"":{""comments_count"":0,""cr...",541305,Alfa lue og hals,https://images4-b.ravelrycache.com/uploads/Gur...,Guri Østereng Halvorsen
2,"{""patterns"": {""279164"":{""comments_count"":0,""cr...",279164,F145 Heaven Ribbed Scarf,https://images4-e.ravelrycache.com/uploads/Ply...,JoAnne Turcotte
3,"{""patterns"": {""934524"":{""comments_count"":1,""cr...",934524,Garter Stitch Bonnet,NaN,Yarnspirations Design Studio
4,"{""patterns"": {""934526"":{""comments_count"":0,""cr...",934526,Baby Turban,NaN,Yarnspirations Design Studio


In [ ]:
# use this if/when pattern ID shows up with .0 on the end
# d['pattern_id'] = d['pattern_id'].astype(str)
# d['pattern_id'] = d['pattern_id'].str.replace('\.0', '') # finally

In [ ]:
d.head()

In [63]:
# construct separate df of URLs to clean up original scrape/clean
# split at comma in d[medium2_urls] due to incorrect regex and expand to new df, urls
# name columns
# extract file extension from original image file to use in construction later

urls = d["medium2_urls"].str.split(",", n = 1, expand = True) 
urls.columns = ['medium_url', 'remainder']
# urls['medium_url'] = urls['medium_url'].str[:-1] # use if pattern_id imports with .0

urls['extension'] = urls['medium_url'].str.extract('(jpg|jpeg|gif|png)', re.IGNORECASE, expand = False)

In [ ]:
# in the 5000 dataset some urls have " on the end for no good reason; need to remove

# urls['medium_url'] = urls.replace(to_replace ="", value = "*\'", regex = True) 
# pat = '\"\''
# rep = '\''

# urls['medium_url'].replace(pat, rep, regex=True, inplace=True)
#urls['medium_url'] = urls['medium_url'].str.replace(pat, rep)

In [64]:
# make new df with concat from d and urls; combine by adding columns (axis = 1)
f = pd.concat([d, urls], axis = 1)

In [65]:
f.dropna(how='all') # drop rows where all values are NaN
f['extension'] = f.extension.fillna('jpg')
f.head(5)

,patterns,pattern_id,pattern_name,medium2_urls,author_name,medium_url,remainder,extension
0,"{""patterns"": {""17016"":{""comments_count"":0,""cre...",17016,13505 Bonnet,https://images4-e.ravelrycache.com/uploads/mam...,Dale Design,https://images4-e.ravelrycache.com/uploads/mam...,"""small2_url"":""https://images4-b.ravelrycache.c...",jpg
1,"{""patterns"": {""541305"":{""comments_count"":0,""cr...",541305,Alfa lue og hals,https://images4-b.ravelrycache.com/uploads/Gur...,Guri Østereng Halvorsen,https://images4-b.ravelrycache.com/uploads/Gur...,None,JPG
2,"{""patterns"": {""279164"":{""comments_count"":0,""cr...",279164,F145 Heaven Ribbed Scarf,https://images4-e.ravelrycache.com/uploads/Ply...,JoAnne Turcotte,https://images4-e.ravelrycache.com/uploads/Ply...,None,jpg
3,"{""patterns"": {""934524"":{""comments_count"":1,""cr...",934524,Garter Stitch Bonnet,NaN,Yarnspirations Design Studio,NaN,NaN,jpg
4,"{""patterns"": {""934526"":{""comments_count"":0,""cr...",934526,Baby Turban,NaN,Yarnspirations Design Studio,NaN,NaN,jpg


In [66]:
# make list of tuples using pattern_id column and medium2_urls column

f['ids_urls'] = list(zip(f.pattern_id, f.medium_url, f.extension))

In [67]:
new_csv_filename = '/Users/ssivek/Insight_Projects/Ravelry_Project/Data/cleaned_csv_files/newcsv' + str(which_file) + '.csv'
f.to_csv(new_csv_filename, index=False)

In [54]:
f.head(10)

,patterns,pattern_id,pattern_name,medium2_urls,author_name,medium_url,remainder,extension,ids_urls
0,"{""patterns"": {""276076"":{""comments_count"":0,""cr...",2776,Hoodwink - Hooded infinity scarf,https://images4-b.ravelrycache.com/uploads/Art...,Artesanitarium Designs,https://images4-b.ravelrycache.com/uploads/Art...,"""small2_url"":""https://images4-e.ravelrycache.c...",jpg,"(2776, https://images4-b.ravelrycache.com/uplo..."
1,"{""patterns"": {""538220"":{""comments_count"":0,""cr...",5382,Adelia Beanie,https://images4-b.ravelrycache.com/uploads/The...,Lauren Klipp,https://images4-b.ravelrycache.com/uploads/The...,None,jpg,"(5382, https://images4-b.ravelrycache.com/uplo..."
2,"{""patterns"": {""931435"":{""comments_count"":0,""cr...",931435,Fox Hat and Mittens,https://images4-b.ravelrycache.com/uploads/luc...,Lucy Chaplin,https://images4-b.ravelrycache.com/uploads/luc...,None,jpeg,"(931435, https://images4-b.ravelrycache.com/up..."
3,"{""patterns"": {""800367"":{""comments_count"":0,""cr...",0367,gavroche hat,https://images4-b.ravelrycache.com/uploads/pal...,Palaluna,https://images4-b.ravelrycache.com/uploads/pal...,"""small2_url"":""https://images4-b.ravelrycache.c...",JPG,"(0367, https://images4-b.ravelrycache.com/uplo..."
4,"{""patterns"": {""13936"":{""comments_count"":0,""cre...",13936,Tam,https://images4-b.ravelrycache.com/uploads/Woo...,Charlene Schurch,https://images4-b.ravelrycache.com/uploads/Woo...,None,jpg,"(13936, https://images4-b.ravelrycache.com/upl..."
5,"{""patterns"": {""276080"":{""comments_count"":3,""cr...",27,Nantaise,https://images4-e.ravelrycache.com/uploads/jad...,Kasia Lubinska,https://images4-e.ravelrycache.com/uploads/jad...,"""small2_url"":""https://images4-b.ravelrycache.c...",jpg,"(27, https://images4-e.ravelrycache.com/upload..."
6,"{""patterns"": {""538224"":{""comments_count"":0,""cr...",538224,Woodland Walk,https://images4-b.ravelrycache.com/uploads/kno...,Lara Simonson,https://images4-b.ravelrycache.com/uploads/kno...,None,jpg,"(538224, https://images4-b.ravelrycache.com/up..."
7,"{""patterns"": {""931443"":{""comments_count"":0,""cr...",931443,Pumpkin Hat,https://images4-e.ravelrycache.com/uploads/luc...,Lucy Chaplin,https://images4-e.ravelrycache.com/uploads/luc...,"""small2_url"":""https://images4-b.ravelrycache.c...",JPG,"(931443, https://images4-e.ravelrycache.com/up..."
8,"{""patterns"": {""13940"":{""comments_count"":2,""cre...",139,Owl Hat,https://images4-b.ravelrycache.com/uploads/iFe...,Ruthie Nussbaum,https://images4-b.ravelrycache.com/uploads/iFe...,None,jpg,"(139, https://images4-b.ravelrycache.com/uploa..."
9,"{""patterns"": {""931451"":{""comments_count"":0,""cr...",931451,Woven Cable Hat and Snood,https://images4-b.ravelrycache.com/uploads/luc...,Lucy Chaplin,https://images4-b.ravelrycache.com/uploads/luc...,"""small2_url"":""https://images4-b.ravelrycache.c...",JPG,"(931451, https://images4-b.ravelrycache.com/up..."


In [ ]:
# f.loc[f['pattern_id'] == 525904]['ids_urls']

In [70]:
# loop through column of tuples (3 items each) 
# make output filename with pattern ID (a) and existing file extension (c)
# URL is just (b)
# append each new tuple of id, url, output filename to new list of queries


img_query = []

for a, b, c in f['ids_urls']:
    try:
        file_ext = c # end of existing filename
        output_file = str(a)[:-2] + "." + c # img output file is ID without .0 + . + file ext
        output_file = str(a) + "." + c # img output file is ID + . + file ext
        url = b 
        img_query.append((output_file, url))
    except:
        if c == 'NaN': # not all filenames had extensions 
            # id_file = id + ".jpg" # is this causing a problem?
            output_file = a
            continue
        if b == 'nan':
            continue
        img_query.append((output_file, url))

In [71]:
# subset of all queries for testing
test = img_query[0:10]
print(test)

[('17016.jpg', 'https://images4-e.ravelrycache.com/uploads/mamaurchin/562837833/146922112_2ee6970b65_z_medium2.jpg"'), ('541305.JPG', 'https://images4-b.ravelrycache.com/uploads/Gurigurimalla/269432850/Alfa-alfa_lue_og_hals_RED_-_Kopi_-_Kopi_medium2.JPG'), ('279164.jpg', 'https://images4-e.ravelrycache.com/uploads/PlymouthYarn/80341071/F145Heaven_medium2.jpg'), ('934524.jpg', nan), ('934526.jpg', nan), ('541313.JPG', 'https://images4-e.ravelrycache.com/uploads/hnick/249279679/IMG_1269_medium2.JPG"'), ('541314.jpg', 'https://images4-e.ravelrycache.com/uploads/veera/269439936/black-and-grey-1_medium2.jpg"'), ('803459.jpg', 'https://images4-b.ravelrycache.com/uploads/sharemypassion/495847426/IMG_20180115_092419_medium2.jpg"'), ('148100.jpg', 'https://images4-b.ravelrycache.com/uploads/fibrosity/21041231/DSCF4573_medium2.jpg"'), ('810984.jpg', 'https://images4-e.ravelrycache.com/uploads/Vanja71/502111548/rubix3_medium2.jpg')]


In [72]:
# for each tuple in query list, a is new filename to use, b is url to download

failed_pattern_filenames = [] # would have to get pattern ids out of these names to follow up

for a, b in img_query:
    try:
        file_path = "/Users/ssivek/Insight_Projects/Ravelry_Project/Data/images_" + str(which_file) + "/"
        file_name = file_path + a
        r = rq.get(b, timeout=(3, 5)) # set connect and read timeouts
        with open(file_name, 'wb+') as g:
            g.write(r.content)
        print("Success:" + a)
    except rq.exceptions.RequestException:
        failed_pattern_filenames.append(a)
        time.sleep(5.2)
        continue 

Success:17016.jpg
Success:541305.JPG
Success:279164.jpg
Success:541313.JPG
Success:541314.jpg
Success:803459.jpg
Success:148100.jpg
Success:810984.jpg
Success:541322.png
Success:279179.jpg
Success:148108.jpg
Success:672394.jpg
Success:934541.jpg
Success:148111.jpg
Success:279184.jpg
Success:410259.jpg
Success:410260.jpg
Success:148117.jpg
Success:541333.jpg
Success:279191.JPG
Success:541335.jpg
Success:17049.png
Success:672406.jpg
Success:803476.jpg
Success:410269.jpg
Success:148126.jpg
Success:541341.jpg
Success:148129.jpg
Success:410273.jpg
Success:410274.jpg
Success:803493.jpg
Success:672422.jpg
Success:410279.jpg
Success:17064.jpg
Success:672423.jpg
Success:672425.jpg
Success:541371.JPG
Success:148156.jpg
Success:279229.JPG
Success:541374.JPG
Success:934589.jpg
Success:672448.JPG
Success:803521.jpg
Success:148162.jpg
Success:410306.jpg
Success:410307.jpg
Success:410309.jpg
Success:672450.JPG
Success:672451.JPG
Success:672452.JPG
Success:803529.jpg
Success:803530.jpg
Success:803531.

Success:148778.jpg
Success:279850.jpg
Success:804134.jpg
Success:148781.jpg
Success:804135.jpg
Success:148783.jpg
Success:410928.jpg
Success:804138.jpg
Success:17714.jpg
Success:17715.jpg
Success:279860.JPG
Success:804146.jpg
Success:17718.jpg
Success:148791.jpg
Success:279864.jpg
Success:410934.jpg
Success:410935.jpg
Success:542006.jpg
Success:673082.jpg
Success:148797.jpg
Success:542014.JPG
Success:17730.jpg
Success:148803.jpg
Success:17732.jpg
Success:148804.JPG
Success:542020.jpg
Success:673101.jpg
Success:542035.jpg
Success:410966.JPG
Success:673111.jpg
Success:804183.jpg
Success:673113.png
Success:148826.jpg
Success:17755.jpg
Success:804184.jpg
Success:17757.jpg
Success:542047.jpg
Success:279904.JPG
Success:279906.jpg
Success:542056.jpg
Success:673128.jpeg
Success:410986.jpg
Success:410987.jpg
Success:542058.jpg
Success:542059.jpg
Success:279918.jpg
Success:542060.jpg
Success:542064.jpg
Success:673130.jpg
Success:804204.jpg
Success:542070.jpg
Success:148858.JPG
Success:804218.jpg

Success:18462.jpg
Success:18463.jpg
Success:411679.jpg
Success:542747.jpg
Success:542756.jpg
Success:411686.jpg
Success:804902.jpg
Success:804903.jpg
Success:149545.jpg
Success:804907.jpg
Success:18481.jpg
Success:18483.jpg
Success:149555.JPG
Success:804915.jpg
Success:804916.jpg
Success:411703.jpg
Success:804924.jpg
Success:804925.jpg
Success:542782.jpg
Success:804927.jpg
Success:18496.jpg
Success:411713.jpg
Success:411715.jpg
Success:149572.jpg
Success:804933.jpg
Success:411720.jpg
Success:280649.jpg
Success:18506.jpg
Success:411722.jpg
Success:804940.png
Success:149582.JPG
Success:411726.JPG
Success:681020.jpg
Success:542802.jpg
Success:18517.jpg
Success:280663.jpg
Success:542807.JPG
Success:280665.jpg
Success:804952.JPG
Success:673883.jpg
Success:149596.jpg
Success:149597.jpg
Success:917716.jpg
Success:411745.jpg
Success:673889.jpeg
Success:18531.jpg
Success:542819.jpg
Success:542824.jpg
Success:280684.jpg
Success:804971.jpg
Success:542834.png
Success:673910.jpg
Success:280701.jpg


In [73]:
# store failed filenames list to file

list_path = "/Users/ssivek/Insight_Projects/Ravelry_Project/Data/failed" + str(which_file) + ".p"

with open(list_path, 'wb') as fp:
    pickle.dump(failed_pattern_filenames, fp)